In [1]:
root_dir ="/media/enc/vera1/sebastian/codes/ARF_STUFF/results/doing/zeroshot_13May"

In [2]:
# !pip install river==0.21.0
import argparse
import datetime
import json
import numpy as np
import os
import joblib 
from pympler import asizeof
from pathlib import Path
import copy
import torch
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import pandas as pd
import csv
from datetime import datetime
#from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import PassiveAggressiveRegressor

import time
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

from scipy.signal import savgol_filter
import random
import pickle
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from river import forest,tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor
import copy
import sys
sys.path.insert(0,'../../') 
from pandas import DataFrame, concat
from xgboost import XGBRegressor
from src.utils import save_timestamps, store_pickle_model, save_predictions
import src.misc as misc
from src.misc import NativeScalerWithGradNormCount as NativeScaler
from src.datasets import load_orangepi_data, normalize_data, series_to_supervised, structure, denormalize_predictions
from src.metrics import symmetric_mean_absolute_percentage_error, mean_absolute_scaled_error, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error, r2_score, calculate_metrics
from src.train import train_xgb_model, test_xgb_model_update, train_river_model, test_river_model, train_sklearn_model, test_sklearn_model, train_sklearn_partial_fit,test_sklearn_partial_fit
from src.memory import bytes_to_mb
import numpy as np
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
import glob 

2024-05-17 16:50:24.219565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 16:50:24.219596: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 16:50:24.220494: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-17 16:50:24.225259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 16:50:24.859213: W tensorflow/compiler/tf2

In [15]:
def process_model_directories(root_dir,model_name):
    # Define lists to store values
    MAE_values = []
    RMSE_values = []
    SMAPE_values = []
    r2_values = []
    MASE_values = []
    training_time_values = []
    inference_time_values = []
    model_memory_values = []
    model_metrics_path = glob.glob(os.path.join(root_dir, "*model*.csv"))[0]# since its a single element, take the first

    model_metrics = pd.read_csv(model_metrics_path)
    model_metrics["Model"] = model_name
    rename = {'mae': 'MAE',
        'rmse':'RMSE',
        'r2':'r2',
        'smape':'SMAPE',
        'mase':'MASE',
        'Inference Time':'Inference_time'
    }
    
    return model_metrics.rename(columns=rename)
    

def arrays_to_dataframe(MAE_array, RMSE_array, SMAPE_array, r2_array, MASE_array,
                        training_time_array, inference_time_array, model_memory_array):
    # Create a DataFrame
    df = pd.DataFrame({
        'MAE': MAE_array,
        'RMSE': RMSE_array,
        'SMAPE': SMAPE_array,
        'r2': r2_array,
        'MASE': MASE_array,
        'Training_time': training_time_array,
        'Training_time_log': np.log(training_time_array),
        'Inference_time': inference_time_array,
        'Inference_time_log': np.log(inference_time_array),
        'Model_memory': model_memory_array
    })
    
    return df

dfs = []  # List to store DataFrames

for subdir in os.listdir(root_dir):
        subdir_path = os.path.join(root_dir, subdir)
        if os.path.isdir(subdir_path) and not subdir_path.endswith(".ipynb_checkpoints"):
            models = os.listdir(subdir_path)
            for testbed in models:
                subdir_path_tb = os.path.join(subdir_path, testbed)
                model_name = subdir_path_tb.split("/")[-1]
                # print("model:",model_name)
                metrics = process_model_directories(subdir_path_tb,model_name)
                dfs.append(metrics)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df

,MAE,RMSE,r2,SMAPE,MASE,Inference_time,Model
0,8.823205,15.134116,0.736951,37.049332,2.128658,1205.137391,model_CL_128_RoPE_False
1,9.837194,19.108541,0.581767,36.023745,3.241959,1142.883972,model_CL_64_RoPE_False
2,9.341400,18.842302,0.576871,32.566130,3.179916,1121.509457,model_CL_64_RoPE_True
3,6.235191,14.533580,0.754402,26.635012,2.242244,1054.365297,model_CL_32_RoPE_False
4,7.116079,14.306186,0.770448,33.164918,1.752418,1207.367312,model_CL_128_RoPE_True
...,...,...,...,...,...,...,...
155,6.230777,14.539863,0.754732,26.578191,2.240287,1095.872958,model_CL_32_RoPE_False
156,7.128440,14.329398,0.769666,33.038440,1.759934,1210.506856,model_CL_128_RoPE_True
157,5.509868,11.553521,0.857807,32.181707,1.167863,1531.349224,model_CL_256_RoPE_True
158,6.249434,14.565983,0.753342,26.658800,2.250206,1128.815229,model_CL_32_RoPE_True


In [16]:
summary = combined_df.groupby(['Model',]).agg({
                                                                     'MAE'  :['mean', 'std'],
                                                                     'RMSE' :['mean', 'std'],
                                                                     'SMAPE':['mean', 'std'],
                                                                     'r2'   :['mean', 'std'],
                                                                     'MASE' :['mean', 'std'],
                                                                     'Inference_time' :['mean'],
                                                                      }).reset_index()
summary = summary.round(3)
summary.to_csv(f'{root_dir}/model_metrics_avg.csv', index=False)
summary

Model    MAE           RMSE          SMAPE            r2  \
                             mean    std    mean    std    mean    std   mean   
0  model_CL_128_RoPE_False  8.839  0.021  15.130  0.033  37.335  0.206  0.737   
1   model_CL_128_RoPE_True  7.115  0.013  14.319  0.042  33.121  0.073  0.770   
2  model_CL_256_RoPE_False  9.651  0.034  14.300  0.044  39.557  0.197  0.746   
3   model_CL_256_RoPE_True  5.502  0.018  11.579  0.043  32.021  0.110  0.857   
4   model_CL_32_RoPE_False  6.248  0.011  14.571  0.019  26.672  0.059  0.753   
5    model_CL_32_RoPE_True  6.246  0.009  14.565  0.014  26.647  0.060  0.753   
6   model_CL_64_RoPE_False  9.824  0.021  19.090  0.023  35.941  0.135  0.583   
7    model_CL_64_RoPE_True  9.350  0.009  18.838  0.017  32.622  0.086  0.577   

           MASE        Inference_time  
     std   mean    std           mean  
0  0.001  2.130  0.007       1200.982  
1  0.001  1.755  0.011       1208.797  
2  0.001  1.956  0.016       1523.016  
3  0.001  1.169  0.007       1533.625  
4  0.001  2.247  0.012       1070.154  
5  0.001  2.251  0.015       1109.569  
6  0.001  3.239  0.020       1137.884  
7  0.001  3.228  0.021       1119.538